# Running program using decorators

In [3]:
import os

os.environ["ENV_GATEWAY_PROVIDER_TOKEN"] = "<YOUR_TOKEN>"
os.environ["ENV_GATEWAY_PROVIDER_HOST"] = "http://localhost:8000"

## Hello, Qiskit!

In [4]:
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

from quantum_serverless import distribute_program, distribute_task, get, save_result


@distribute_program()  # or define provider explicitly @distribute_program(provider=Provider(...))
def hello_qiskit():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.measure_all()
    circuit.draw()

    sampler = Sampler()
    quasi_dists = sampler.run(circuit).result().quasi_dists

    save_result(quasi_dists)
    
job = hello_qiskit()
job.result()

'[{"0": 0.4999999999999999, "3": 0.4999999999999999}]'

## Program with distributed tasks

In [6]:
from quantum_serverless import get_arguments, save_result, distribute_task, get

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit.circuit.random import random_circuit

@distribute_task(target={"cpu": 2})
def distributed_sample(circuit: QuantumCircuit):
    """Distributed task that returns quasi distribution for given circuit."""
    return Sampler().run(circuit).result().quasi_dists[0]


@distribute_program()
def program_with_distributed_tasks(circuits):
    sample_task_references = [
        distributed_sample(circuit)
        for circuit in circuits
    ]
    results = get(sample_task_references)
    print(results)


circuits = []
for _ in range(3):
    circuit = random_circuit(2, 2)
    circuit.measure_all()
    circuits.append(circuit)

job = program_with_distributed_tasks(circuits=circuits)
job.result()
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
2023-08-24 12:38:05,264	INFO worker.py:1313 -- Using address 172.19.0.3:6379 set in the environment variable RAY_ADDRESS
2023-08-24 12:38:05,264	INFO worker.py:1431 -- Connecting to existing Ray cluster at address: 172.19.0.3:6379...
2023-08-24 12:38:05,279	INFO worker.py:1612 -- Connected to Ray cluster. View the dashboard at 172.19.0.3:8265 
(pid=5596) OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
[{0: 0.7239603189136047, 3: 0.2760396810863955}, {0: 0.9215567523267272, 2: 0.0784432476732726}, {0: 0.0158356782806968, 1: 0.9841643217193032}]
(pid=5597) OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k [repeated 5x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.